In [1]:
import tensorflow as tf

In [99]:
def pretty_print_guess_tensor(const_guess, operand_guess, operator_guess):
    # TODO: const_guess

    s = []

    for t in operand_guess:
        s += [f'x_{tf.argmax(t)}']

    operator_lookup = ['+','-', '*','/']
    result = s[::]
    for i, op_one_hot in enumerate(operator_guess):
        operators = tf.argmax(op_one_hot,axis=-1)
        left = result[::2]
        right = (result[1:] + result[:1])[::2]
        ops = operators[:len(left)]
        result = []
        for l, op, r in zip(left, ops, right):
            result += [f'({l} {operator_lookup[op]} {r})']


    return ' '.join(result)

NUM_LEAVES = 8
NUM_OPERATORS = 4
v1 = tf.range(NUM_LEAVES)
v2 = tf.range(NUM_OPERATORS)

cgv = tf.one_hot(v1 // 2, NUM_LEAVES//2, dtype=tf.float32)
const_guess = tf.concat([cgv, cgv],axis=1)
operand_guess = tf.one_hot(v1, NUM_LEAVES)
ogv = tf.expand_dims(tf.one_hot(v2, NUM_OPERATORS), axis=0)
operator_guess = tf.concat([ogv,ogv,ogv], axis=0)

pretty_print_guess_tensor(const_guess, operand_guess, operator_guess)

'(((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7)))'